In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
import torchvision.transforms.v2 as transforms
import torch.nn as nn
from kvasir_capsule_dataset import get_dataloader
from trainer import train_kc_model
import gc
from custom_transforms import GaussianBlur, RandomChoiceExtended
import os

with_gpu = torch.cuda.is_available()

if with_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print('We are now using %s.' % device)

train_path = r"C:\Users\JadHa\Desktop\Uni\DLMB\DLMI-Project\kvasir-capsule-labeled-images\dataset_train.csv"
val_path = r"C:\Users\JadHa\Desktop\Uni\DLMB\DLMI-Project\kvasir-capsule-labeled-images\dataset_test.csv"
dataset_path = r"C:\Users\JadHa\Desktop\Uni\DLMB\DLMI-Project\kvasir-capsule-labeled-images\labelled_images"

c:\Users\JadHa\anaconda3\envs\pytorch\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\JadHa\anaconda3\envs\pytorch\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you

We are now using cuda.


# Training with Contrastive Learning (SimCLR)

In [11]:
from simclr import SimCLR
from model import ResNetSimCLR
transforms_list = [transforms.Compose([transforms.RandomResizedCrop(size=96),transforms.Resize(size=[336, 336])]),
                   transforms.RandomRotation(degrees=360),
                   GaussianBlur(kernel_size=3),
                   transforms.ColorJitter(0.1, 0.1, 0.003, 0.003)]

transform = RandomChoiceExtended(transforms_list, min_transforms=0, max_transforms=3)

dataset_len = 39201

train_loader = get_dataloader(csv_path=train_path, 
                              dataset_path=dataset_path, 
                              batch_size=64, 
                              use_preloaded=True, 
                              start_idx=0, 
                              end_idx=int(2**13)+4096, 
                              shuffle=True, 
                              transforms=transform, 
                              drop_data_till_balanced=False, 
                              simclr=True, 
                              pin_memory=True, 
                              drop_last=True)

In [43]:
model = ResNetSimCLR(base_model="resnet18", out_dim=128)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                        last_epoch=-1)

args = {
    "device" : torch.device("cuda"),
    "batch_size" : 64,
    "n_views" : 2,
    "temperature" : 0.07,
    "fp16_precision" : True,
    "epochs" : 200,
    "log_every_n_steps" : 100,
    "arch" : "resnet18",
    "save_every_n_epochs" : 5
}

#  It’s a no-op if the 'gpu_index' argument is a negative integer or None.
with torch.cuda.device(0):
    torch.cuda.empty_cache()
    gc.collect()
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, args=args)
    simclr.train(train_loader)

c:\Users\JadHa\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\JadHa\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  0%|          | 0/192 [00:00<?, ?it/s]c:\Users\JadHa\anaconda3\envs\pytorch\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antial

Epoch: 0	Loss: 2.2531490325927734


100%|██████████| 192/192 [07:17<00:00,  2.28s/it]


Epoch: 1	Loss: 1.7051080465316772


100%|██████████| 192/192 [07:16<00:00,  2.27s/it]


...

Epoch: 197	Loss: 0.20794028043746948


100%|██████████| 192/192 [06:56<00:00,  2.17s/it]


Epoch: 198	Loss: 0.527816116809845


100%|██████████| 192/192 [06:56<00:00,  2.17s/it]


Epoch: 199	Loss: 0.3368372619152069


# Finetuning pretrained model

In [4]:
from model import ResNetSimCLR

dataset_len = 39201
train_loader = get_dataloader(csv_path=train_path, 
                              dataset_path=dataset_path, 
                              batch_size=128, 
                              use_preloaded=True, 
                              start_idx=0, 
                              end_idx=dataset_len-1, 
                              shuffle=True, 
                              transforms=None, 
                              drop_data_till_balanced=True, 
                              simclr=False)

val_loader = get_dataloader(csv_path=val_path, dataset_path=dataset_path, batch_size=128, shuffle=True, use_preloaded=True, start_idx=0, end_idx=8047, simclr=False)

In [18]:
chkpoint = torch.load(os.path.join(r"runs\Jan25_03-08-13_DESKTOP-K6APQCL", "checkpoint_0200.pt"))

model = ResNetSimCLR(base_model="resnet18", out_dim=128)
model.load_state_dict(chkpoint["state_dict"])

for param in model.parameters(): # Freeze all parameters
    param.requires_grad = False

model.backbone.fc = nn.Linear(512, 2) # Replace fc layer from SimCLR with a new one
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

scaler = torch.cuda.amp.GradScaler()
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1, 9]).to(device))

model_name = "ResNet_SimCLR"

In [ ]:
chkpoint = torch.load(os.path.join("saved_models", "%s.pt"%model_name))
model.load_state_dict(chkpoint["model_state_dict"])
optimizer.load_state_dict(chkpoint["optimizer"])
scaler.load_state_dict(chkpoint["scaler"])

In [19]:
torch.cuda.empty_cache()
gc.collect()
train_kc_model(model, optimizer, criterion, train_loader, val_loader, scaler, model_name=model_name, epochs=40, device=device)

Training :
Epoch : 1, Loss : 1.821, Accuracy : 0.500, Precision : 0.500, Recall : 1.000
Validating :
Epoch : 1, Loss : 1.863, Accuracy : 0.004, Precision : 0.004, Recall : 1.000
Model is stored at folder:saved_models/ResNet_SimCLR.pt
Training :
Epoch : 2, Loss : 1.570, Accuracy : 0.500, Precision : 0.500, Recall : 1.000
Validating :
Epoch : 2, Loss : 1.933, Accuracy : 0.004, Precision : 0.004, Recall : 1.000
Model is stored at folder:saved_models/ResNet_SimCLR.pt
Training :
Epoch : 3, Loss : 1.194, Accuracy : 0.502, Precision : 0.501, Recall : 1.000
Validating :
Epoch : 3, Loss : 1.612, Accuracy : 0.009, Precision : 0.004, Recall : 1.000
Model is stored at folder:saved_models/ResNet_SimCLR.pt
Training :
Epoch : 4, Loss : 0.965, Accuracy : 0.535, Precision : 0.518, Recall : 1.000
Validating :
Epoch : 4, Loss : 1.056, Accuracy : 0.205, Precision : 0.005, Recall : 1.000
Model is stored at folder:saved_models/ResNet_SimCLR.pt
Training :
Epoch : 5, Loss : 0.849, Accuracy : 0.685, Precision 

# Training with normal supervised Learning

In [ ]:
transforms_list = [transforms.Compose([transforms.RandomResizedCrop(size=96),transforms.Resize(size=[336, 336])]),
                   transforms.RandomRotation(degrees=360),
                   GaussianBlur(kernel_size=3),
                   transforms.ColorJitter(0.1, 0.1, 0.003, 0.003)]

transform = RandomChoiceExtended(transforms_list, min_transforms=0, max_transforms=3)

dataset_len = 39201

train_loader = get_dataloader(csv_path=train_path, dataset_path=dataset_path, batch_size=128, use_preloaded=True, start_idx=4096, end_idx=int(2**13)+4096, shuffle=True, transforms=transform, drop_data_till_balanced=False, simclr=True)
val_loader = get_dataloader(csv_path=val_path, dataset_path=dataset_path, batch_size=128, shuffle=True, use_preloaded=True, start_idx=0, end_idx=8047, simclr=False)

In [ ]:
resnet = resnet18().to(device) # weights=ResNet18_Weights.IMAGENET1K_V1

resnet.fc = nn.Linear(512, 2).to(device)

optimizer = optim.AdamW(params=resnet.parameters(), lr=3e-4)
scaler = torch.cuda.amp.GradScaler()

criterion = nn.CrossEntropyLoss() # weight=torch.tensor([1, 20]).to(device)

In [ ]:
torch.cuda.empty_cache()
gc.collect()
train_kc_model(resnet, optimizer, criterion, train_loader, val_loader, scaler, model_name="Resnet_AMP", epochs=40, device=device)

c:\Users\JadHa\anaconda3\envs\pytorch\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Training :
Epoch : 1, Loss : 0.017, Accuracy : 0.968, Precision : 0.657, Recall : 0.804
Validating :
Epoch : 1, Loss : 0.065, Accuracy : 0.949, Precision : 0.008, Recall : 0.088
Model is stored at folder:saved_models/Resnet_AMP.pt
Training :
Epoch : 2, Loss : 0.023, Accuracy : 0.989, Precision : 0.921, Recall : 0.851
Validating :
Epoch : 2, Loss : 0.215, Accuracy : 0.940, Precision : 0.007, Recall : 0.088
Model is stored at folder:saved_models/Resnet_AMP.pt
Training :
Epoch : 3, Loss : 0.004, Accuracy : 0.991, Precision : 0.942, Recall : 0.884
Validating :
Epoch : 3, Loss : 0.372, Accuracy : 0.941, Precision : 0.007, Recall : 0.088
Model is stored at folder:saved_models/Resnet_AMP.pt
Training :
Epoch : 4, Loss : 0.007, Accuracy : 0.993, Precision : 0.951, Recall : 0.917
Validating :
Epoch : 4, Loss : 0.794, Accuracy : 0.941, Precision : 0.007, Recall : 0.088
Model is stored at folder:saved_models/Resnet_AMP.pt
Training :
Epoch : 5, Loss : 0.008, Accuracy : 0.994, Precision : 0.954, Rec

KeyboardInterrupt: 

# Training an XGBoost Classifier

In [13]:
from kvasir_capsule_dataset import KC_Dataset_Features
dataset_len = 39201
train_dataset = KC_Dataset_Features(start_idx=0, end_idx=dataset_len, csv_path=train_path, dataset_path=dataset_path)
val_dataset = KC_Dataset_Features(start_idx=0, end_idx=8047, csv_path=val_path, dataset_path=dataset_path)

100%|██████████| 8047/8047 [23:54<00:00,  5.61it/s]


In [22]:
import xgboost as xgb

X_train, y_train = (train_dataset.x, train_dataset.y)
X_test, y_test = (val_dataset.x, val_dataset.y)

# Use "hist" for constructing the trees, with early stopping enabled.
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2, scale_pos_weight=1/100)
# Fit the model, test sets are used for early stopping.
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
# Save model into JSON format.
clf.save_model("xgboost_3.json")

preds = clf.predict(X_test)

[0]	validation_0-logloss:0.58344
[1]	validation_0-logloss:0.40206
[2]	validation_0-logloss:0.29162
[3]	validation_0-logloss:0.25121
[4]	validation_0-logloss:0.22556
[5]	validation_0-logloss:0.20624
[6]	validation_0-logloss:0.19864
[7]	validation_0-logloss:0.19456
[8]	validation_0-logloss:0.19351
[9]	validation_0-logloss:0.19281
[10]	validation_0-logloss:0.19485
[11]	validation_0-logloss:0.19263
[12]	validation_0-logloss:0.18566
[13]	validation_0-logloss:0.18701
[14]	validation_0-logloss:0.18977


In [18]:
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
clf.load_model("clf.json")
preds = clf.predict(X_test)

In [23]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(np.argmax(y_test, axis=1), np.argmax(preds, axis=1), target_names=["normal", "blood"]))
print(np.sum(np.logical_and(np.argmax(y_test, axis=1) == 1, np.argmax(preds, axis=1) == 1)) / np.sum(np.argmax(y_test, axis=1)))
print(np.sum(np.logical_and(np.argmax(y_test, axis=1) == 1, np.argmax(preds, axis=1) == 1)) / np.sum(np.argmax(preds, axis=1)))

              precision    recall  f1-score   support

      normal       1.00      0.97      0.98      8013
       blood       0.00      0.00      0.00        34

    accuracy                           0.97      8047
   macro avg       0.50      0.49      0.49      8047
weighted avg       0.99      0.97      0.98      8047

0.0
0.0
